# Домашнее задание №6

###### Задание 1

In [5]:
geo_data = {

    'Центр': ['москва', 'тула', 'ярославль'],

    'Северо-Запад': ['петербург', 'псков', 'мурманск'],

    'Дальний Восток': ['владивосток', 'сахалин', 'хабаровск']

}

In [6]:
import pandas as pd
keywords = pd.read_csv('c:\\temp\\keywords.csv')
def keyword_region(row):
   for i in geo_data:
       for j in geo_data[i]:
           if j in row['keyword']:
               return i
   else:
        return 'undefined'

In [7]:
keywords['region'] = keywords.apply(keyword_region, axis=1)
keywords.head()

,keyword,shows,region
0,вк,64292779,undefined
1,одноклассники,63810309,undefined
2,порно,41747114,undefined
3,ютуб,39995567,undefined
4,вконтакте,21014195,undefined


In [8]:
region = keywords[keywords['keyword'].isin(['владивосток', 'сахалин', 'хабаровск'])]
region.head()

,keyword,shows,region
5858,владивосток,43038,Дальний Восток
9084,хабаровск,29655,Дальний Восток
21445,сахалин,14494,Дальний Восток


In [141]:
region = keywords[keywords['keyword'].isin(['петербург', 'псков', 'мурманск'])]
region.head()

,keyword,shows,region
8977,мурманск,29984,Северо-Запад
11222,псков,24885,Северо-Запад
20823,петербург,14859,Северо-Запад


###### Задание 2

In [9]:
ratings = pd.read_csv('c:\\temp\\ml-latest-small\\ratings.csv')
movies = pd.read_csv('c:\\temp\\ml-latest-small\\movies.csv')
ratings_avg=ratings.groupby('movieId').mean()[['rating']].sort_values('rating', ascending=False)
ratings_avg.head()

,rating
movieId,
88448,5.0
100556,5.0
143031,5.0
143511,5.0
143559,5.0


In [11]:
def class_rate(row):
    if row['rating'] <= 2.0:
        return 'низкий рейтинг'
    if row['rating'] >= 4.5:
        return 'высокий рейтинг'
    else:
        return 'средний рейтинг'

In [12]:
ratings_avg['class'] = ratings_avg.apply(class_rate, axis=1)
ratings_avg.head()

,rating,class
movieId,,
88448,5.0,высокий рейтинг
100556,5.0,высокий рейтинг
143031,5.0,высокий рейтинг
143511,5.0,высокий рейтинг
143559,5.0,высокий рейтинг


In [13]:
rated = movies.merge(ratings_avg.loc[:, ['class', 'rating']], on='movieId', how='left')
rated.head()

,movieId,title,genres,class,rating
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,средний рейтинг,3.920930
1,2,Jumanji (1995),Adventure|Children|Fantasy,средний рейтинг,3.431818
2,3,Grumpier Old Men (1995),Comedy|Romance,средний рейтинг,3.259615
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance,средний рейтинг,2.357143
4,5,Father of the Bride Part II (1995),Comedy,средний рейтинг,3.071429


###### Задание 3

In [10]:
import numpy as np
ratings.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [18]:
ratings_counted=ratings.groupby('userId').count()[['rating']]
ratings_filtered=ratings_counted[ratings_counted['rating'] > 100]
ratings_filtered.head()

,rating
userId,
1,232
4,216
6,314
7,152
10,140


In [56]:
ratings_timestamp=ratings.groupby('userId').agg(['min', 'max'])['timestamp']
ratings_timestamp['min'] = ratings_timestamp['min'].astype(int)
ratings_timestamp['max'] = ratings_timestamp['max'].astype(int)
ratings_timestamp.head()

,min,max
userId,,
1,964980499,965719662
2,1445714835,1445715340
3,1306463323,1306464293
4,945078428,1007574542
5,847434747,847435337


In [64]:
ratings_timestamp['lifetime'] = ratings_timestamp.apply(lambda x: ratings_timestamp['max'].sub(ratings_timestamp['min'], axis=0), axis=0)
ratings_timestamp.head()

,min,max,lifetime
userId,,,
1,964980499,965719662,739163
2,1445714835,1445715340,505
3,1306463323,1306464293,970
4,945078428,1007574542,62496114
5,847434747,847435337,590


In [140]:
data = ratings_filtered.merge(ratings_timestamp.loc[:, ['lifetime']], on='userId', how='left')
data['lifetime'] = data['lifetime'] / 86400
data.mean()[['lifetime']].head()

lifetime    517.724164
dtype: float64

###### Задание 4

In [88]:
years = np.arange( 1950, 2011 )
years

array([1950, 1951, 1952, 1953, 1954, 1955, 1956, 1957, 1958, 1959, 1960,
       1961, 1962, 1963, 1964, 1965, 1966, 1967, 1968, 1969, 1970, 1971,
       1972, 1973, 1974, 1975, 1976, 1977, 1978, 1979, 1980, 1981, 1982,
       1983, 1984, 1985, 1986, 1987, 1988, 1989, 1990, 1991, 1992, 1993,
       1994, 1995, 1996, 1997, 1998, 1999, 2000, 2001, 2002, 2003, 2004,
       2005, 2006, 2007, 2008, 2009, 2010])

In [142]:
def production_year(row):
    for i in years:
        if (row['title'].find(str(i))>0):
            return i
    else:
        return 1900

In [143]:
movies['year'] = movies.apply(production_year, axis=1)
movies.head()

,movieId,title,genres,year
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1995
1,2,Jumanji (1995),Adventure|Children|Fantasy,1995
2,3,Grumpier Old Men (1995),Comedy|Romance,1995
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance,1995
4,5,Father of the Bride Part II (1995),Comedy,1995


In [138]:
films = movies.merge(ratings_avg.loc[:, ['rating']], on='movieId', how='left')
films_by_year = films.groupby('year').mean()[['rating']].sort_values('rating', ascending=False)
films_by_year.head()

,rating
year,
1971,3.811243
1957,3.767680
1951,3.718044
1952,3.700104
1969,3.682212
